In [1]:
import tensorflow as tf
import numpy as np
import os

tf.logging.set_verbosity(tf.logging.INFO)

########## Hyperparameter ##########
BATCH_SIZE = 200
EPOCH_BOND = 1
EARLY_STOP_CHECK_EPOCH = 10
########## Hyperparameter ##########

def weight_variable(shape, w=0.1):
    initial = tf.truncated_normal(shape, stddev=w) #Outputs random values from a truncated normal distribution.
    return tf.Variable(initial)

def bias_variable(shape, w=0.1):
    initial = tf.constant(w, shape=shape)
    return tf.Variable(initial)

def cnn_model(features, labels, mode):
    # Input Layer
    # input layer shape should be [batch_size, image_width, image_height, channels] for conv2d
    # set batch_size = -1 means batch_size = the number of input
    print('input data shape: ', features)
    input_layer = tf.reshape(features, [-1, 32, 32, 3])
    print('input layer shape: ',input_layer.shape)
    # conv1
    with tf.variable_scope('conv1') as scope:
        kernel = weight_variable(shape=[5, 5, 3, 64]) #shape=[filter_height * filter_width * in_channels, output_channels]
        conv = tf.nn.conv2d(input_layer, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.0)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv1 = tf.nn.relu(pre_activation, name=scope.name)
       
    # pool1
    pool1 = tf.nn.max_pool(conv1, ksize=[1, 3, 3, 1], strides=[1, 2, 2, 1],
                         padding='SAME', name='pool1')
    print('pool1 shape: ', pool1)
    # norm1
    norm1 = tf.nn.lrn(pool1, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75, name='norm1')

    # conv2
    with tf.variable_scope('conv2') as scope:
        kernel = weight_variable(shape=[5, 5, 64, 64])
        conv = tf.nn.conv2d(norm1, kernel, [1, 1, 1, 1], padding='SAME')
        biases = bias_variable(shape=[64], w=0.1)
        pre_activation = tf.nn.bias_add(conv, biases)
        conv2 = tf.nn.relu(pre_activation, name=scope.name)

    # norm2
    norm2 = tf.nn.lrn(conv2, 4, bias=1.0, alpha=0.001 / 9.0, beta=0.75,
                    name='norm2')
    # pool2
    pool2 = tf.nn.max_pool(norm2, ksize=[1, 3, 3, 1],
                         strides=[1, 2, 2, 1], padding='SAME', name='pool2')
    print('pool2 shape: ', pool2)
    
    pool2_flat = tf.reshape(pool2, [-1, 8*8*64])
    dense1 = tf.layers.dense(
        inputs=pool2_flat,
        units=1024, # number of neurons in the dense layer
        activation=tf.nn.relu,
        name='dense1')
    dropout1 = tf.layers.dropout(
        inputs=dense1,
        rate=0.4,
        training= mode=='TRAIN',
        name='dropout1')
    dense2 = tf.layers.dense(
        inputs=dropout1,
        units=1024, # number of neurons in the dense layer
        activation=tf.nn.relu,
        name='dense2')
    dropout2 = tf.layers.dropout(
        inputs=dense2,
        rate=0.4,
        training= mode=='TRAIN',
        name='dropout2')

    # Logits Layer
    logits = tf.layers.dense(inputs=dropout2, units=100, name='logits')
    
    return logits

def train(X_train, y_train, X_validate, y_validate, optimizer, epoch_bound, stop_threshold, batch_size, testing=False):

    global saver
    global predictions
    
    early_stop = 0
    winner_accuracy = 0.0
    
    for epoch in range(epoch_bound):

        # randomize training set
        indices_training = np.random.permutation(X_train.shape[0])
        X_train, y_train = X_train[indices_training,:], y_train[indices_training]

        # split training set into multiple mini-batches and start training
        total_batches = int(X_train.shape[0] / batch_size)
        for batch in range(total_batches):
            if batch == total_batches - 1:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size:], 
                                               y: y_train[batch*batch_size:], 
                                               mode:'TRAIN'})
            else:
                sess.run(optimizer, feed_dict={x: X_train[batch*batch_size : (batch+1)*batch_size], 
                                               y: y_train[batch*batch_size : (batch+1)*batch_size], 
                                               mode:'TRAIN'})

        # compute accuracy
        predictions = sess.run(predictions, feed_dict={x:X_validate, y:y_validate, mode:'EVAL'})
        cur_accuracy = sess.run(tf.metrics.accuracy(labels=y_validate, predictions=predictions['classes']))
        
        print('Epoch: ', epoch, 'Accuracy: ', cur_accuracy)
        
        # If the accuracy rate does not increase for many times, it will early stop epochs-loop 
        if winner_accuracy < cur_accuracy:
            winner_accuracy = cur_accuracy
            # save best model in testing phase
            if testing == True:
                save_path = saver.save(sess, "./saved_model/cifar-100.ckpt")
        else:
            early_stop += 1
        if early_stop == stop_threshold:
            break

    return winner_accuracy

def unpickle(file):
        import pickle
        with open(file, 'rb') as fo:
            dict = pickle.load(fo, encoding='bytes')
            return dict # return dic keys: [b'filenames', b'batch_label', b'fine_labels', b'coarse_labels', b'data']


########## Data ##########
# Data format:
# data -- a 10000x3072 numpy array of uint8s. 
#         Each row of the array stores a 32x32 colour image. 
#         The first 1024 entries contain the red channel values, 
#         the next 1024 the green, and the final 1024 the blue. 
#         The image is stored in row-major order, so that the first 32 entries of the array are the red channel values of the first row of the image.
# labels -- a list of 10000 numbers in the range 0-99. The number at index i indicates the label of the ith image in the array data.

# Load training data
train_set = unpickle('./Data/cifar-100/train')
train_data = np.asarray(train_set[b'data'], dtype=np.float32) # shape (50000, 3072) 50000 images of 32x32x3 values
train_labels = np.asarray(train_set[b'fine_labels'], dtype=np.int32)

# Load testing data
test_set = unpickle('./Data/cifar-100/test')
eval_data =np.asarray(test_set[b'data'], dtype=np.float32) # shape (10000, 3072) 50000 images of 32x32x3 values
eval_labels = np.asarray(test_set[b'fine_labels'], dtype=np.int32)

print('Train Data shape: ',train_data.shape)
print('Train Label shape: ', train_labels.shape)

########## Data ##########

########## CNN classifier ##########
x = tf.placeholder(tf.float32, [None, train_data.shape[1]])
y = tf.placeholder(tf.int32, [None])
mode = tf.placeholder(tf.string, name='mode')

logits = cnn_model(x, y, mode)

# Calculate Loss (for both TRAIN and EVAL modes)
onehot_labels = tf.one_hot(indices=tf.cast(y, tf.int32), depth=100)
loss = tf.losses.softmax_cross_entropy(
    onehot_labels=onehot_labels, logits=logits)

# Training iteration (for TRAIN)
optimizer = tf.train.GradientDescentOptimizer(learning_rate=0.001)
train_op = optimizer.minimize(
            loss=loss,
            global_step=tf.train.get_global_step())

# For prediction (for EVAL)
probabilities = tf.nn.softmax(logits, name="softmax_tensor")
predictions = {
  # Generate predictions (for PREDICT and EVAL mode)
  "classes": tf.argmax(input=probabilities, axis=1),
  # Add `softmax_tensor` to the graph. It is used for PREDICT and by the
  # `logging_hook`.
  "probabilities": probabilities
}

########## CNN classifier ##########

########## Train ##########
print("########## Start training ##########")
sess = tf.Session()
init = tf.global_variables_initializer()
sess.run(init)

# init saver to save model
saver = tf.train.Saver()
# train
indices_training = np.random.permutation(train_data.shape[0])
validate_indices = indices_training[:BATCH_SIZE]
train_indices = indices_training[BATCH_SIZE:]
validate_data, validate_labels = train_data[validate_indices], train_labels[validate_indices]
train_data, train_labels = train_data[train_indices], train_labels[train_indices]
print('validate data: ',validate_data.shape)
print('train data: ',train_data.shape)
winner_accuracy = train(train_data, train_labels, validate_data, validate_labels
                        , train_op, EPOCH_BOND, EARLY_STOP_CHECK_EPOCH, BATCH_SIZE, testing=True)
########## Train ##########

########## Evaluate ##########
# Evaluate the model and print results
print("########## Start evaluating ##########")

saver.restore(sess, "./saved_model/cifar-100.ckpt")
one_hot_eval_labels = tf.one_hot(indices=tf.cast(eval_labels, tf.int32), depth=100)
sess.run(predictions, feed_dic={x:eval_data, y:one_hot_eval_labels.eval(), mode:'EVAL'})

predic_labels = predictions['classes']
sess.run(predic_labels,feed_dic={x:eval_data, y:eval_labels, mode:'EVAL'})
test_accuracy = tf.metrics.accuracy(labels=one_hot_eval_labels, predictions=predic_labels)
print(eval_results)
########## Evaluate ##########



/Users/jean/tensorflow/lib/python3.6/importlib/_bootstrap.py:219: RuntimeWarning: compiletime version 3.5 of module 'tensorflow.python.framework.fast_tensor_util' does not match runtime version 3.6
  return f(*args, **kwds)


Train Data shape:  (50000, 3072)
Train Label shape:  (50000,)
input data shape:  Tensor("Placeholder:0", shape=(?, 3072), dtype=float32)
input layer shape:  (?, 32, 32, 3)
pool1 shape:  Tensor("pool1:0", shape=(?, 16, 16, 64), dtype=float32)
pool2 shape:  Tensor("pool2:0", shape=(?, 8, 8, 64), dtype=float32)
########## Start training ##########
validate data:  (200, 3072)
train data:  (49800, 3072)


FailedPreconditionError: Attempting to use uninitialized value accuracy/total
	 [[Node: accuracy/total/read = Identity[T=DT_FLOAT, _class=["loc:@accuracy/total"], _device="/job:localhost/replica:0/task:0/device:CPU:0"](accuracy/total)]]

Caused by op 'accuracy/total/read', defined at:
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/local/Cellar/python3/3.6.3/Frameworks/Python.framework/Versions/3.6/lib/python3.6/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/local/lib/python3.6/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/local/lib/python3.6/site-packages/ipykernel/kernelapp.py", line 477, in start
    ioloop.IOLoop.instance().start()
  File "/usr/loca